In [1]:
from pathlib import Path
import pandas as pd
from PyDI.io import load_parquet

ROOT = Path.cwd().parent

DATA_DIR = ROOT / "parquet"
OUTPUT_DIR = ROOT / "output"
MLDS_DIR = ROOT / "ml-datasets"
BLOCK_EVAL_DIR = OUTPUT_DIR / "blocking_evaluation"
CORR_DIR = OUTPUT_DIR / "correspondences"


metacritic = load_parquet(DATA_DIR / "df_metacritic.parquet", name="metacritic")
playtime = load_parquet(DATA_DIR / "df_playtime.parquet", name="playtime")
sales= load_parquet(DATA_DIR / "df_videogamesales.parquet", name="videogamesales")

In [2]:
def add_row_id(df, dataset_name, id_col=None):
    if id_col is None:
        id_col = f"{dataset_name}_row_id"
    df = df.copy()
    df.insert(0, id_col, [f"{dataset_name}_{i+1}" for i in range(len(df))])
    return df

metacritic = add_row_id(metacritic, "metacritic")
sales = add_row_id(sales, "sales")
playtime = add_row_id(playtime, "playtime")


In [3]:
display(sales.head(2))
display(metacritic.head(2))
display(playtime.head(2))

,sales_row_id,title,platform,release_year,publisher,genres,na_sales_mil,eu_sales_mil,jp_sales_mil,other_sales_mil,global_sales_mil
0,sales_1,Wii Sports,Wii,2006.0,Nintendo,[Sports],41.49,29.02,3.77,8.46,82.74
1,sales_2,Super Mario Bros.,NES,1985.0,Nintendo,[Platform],29.08,3.58,6.81,0.77,40.24


,metacritic_row_id,title,platform,release_year,developer,genres,critic_score,user_score,esrb_rating
0,metacritic_1,Portal 2,Xbox 360,2011,Valve Software,"[Action, Shooter, First-Person, Sci-Fi, Arcade]",95.0,8.8,E10+
1,metacritic_2,Metal Gear Solid V: The Phantom Pain,Xbox One,2015,Konami,"[Modern, Action Adventure, Open-World]",95.0,7.5,M


,playtime_row_id,title,platform,release_year,developer,publisher,genres,main_story_hour,main_plus_sides_hour,completionist_hour
0,playtime_1,688(I) Hunter/Killer,PC,1997.0,Sonalysts,Electronic Arts,[Simulation],10.62,35.37,15.83
1,playtime_2,'Splosion Man,Xbox 360,2009.0,Twisted Pixel Games,Microsoft Games Studios,[Action],7.60,9.23,18.77


In [36]:
key_cols = ["title", "platform", "release_year"]

def build_positive_pairs(left_df, right_df, left_tag, right_tag):
    """
    Create positive pairs where title/platform/release_year match exactly.
    `left_tag`/`right_tag` control the suffixes (use 'p', 'm', 's', etc.).
    """
    left = (
        left_df[[f"{left_tag}_row_id"] + key_cols]
        .dropna(subset=key_cols)
        .rename(columns={
            "title": f"title_{left_tag}",
            "platform": f"platform_{left_tag}",
            "release_year": f"release_year_{left_tag}",
        })
    )

    right = (
        right_df[[f"{right_tag}_row_id"] + key_cols]
        .dropna(subset=key_cols)
        .rename(columns={
            "title": f"title_{right_tag}",
            "platform": f"platform_{right_tag}",
            "release_year": f"release_year_{right_tag}",
        })
    )

    pairs = left.merge(
        right,
        left_on=[f"title_{left_tag}", f"platform_{left_tag}", f"release_year_{left_tag}"],
        right_on=[f"title_{right_tag}", f"platform_{right_tag}", f"release_year_{right_tag}"],
        how="inner"
    )

    ordered_cols = [
        f"{left_tag}_row_id",
        f"{right_tag}_row_id",
        f"title_{left_tag}",
        f"title_{right_tag}",
        f"platform_{left_tag}",
        f"platform_{right_tag}",
        f"release_year_{left_tag}",
        f"release_year_{right_tag}",
        "label",
    ]
    return pairs.assign(label=1)[ordered_cols]

def build_negative_pairs_strict(left_df, right_df, left_tag, right_tag,
                                size=None, random_state=0):
    """
    Create *strong* negative pairs:
    - same platform
    - same release_year
    - different title  -> clearly not the same entity
    We also make sure we don't accidentally include any positive pair.
    """
    left = (
        left_df[[f"{left_tag}_row_id"] + key_cols]
        .dropna(subset=key_cols)
        .rename(columns={
            "title": f"title_{left_tag}",
            "platform": f"platform_{left_tag}",
            "release_year": f"release_year_{left_tag}",
        })
    )

    right = (
        right_df[[f"{right_tag}_row_id"] + key_cols]
        .dropna(subset=key_cols)
        .rename(columns={
            "title": f"title_{right_tag}",
            "platform": f"platform_{right_tag}",
            "release_year": f"release_year_{right_tag}",
        })
    )


    candidates = left.merge(
        right,
        left_on=[f"platform_{left_tag}", f"release_year_{left_tag}"],
        right_on=[f"platform_{right_tag}", f"release_year_{right_tag}"],
        how="inner",
    )

    
    candidates = candidates[
        candidates[f"title_{left_tag}"] != candidates[f"title_{right_tag}"]
    ].copy()

    # Drop duplicates
    candidates = candidates.drop_duplicates(
        subset=[f"{left_tag}_row_id", f"{right_tag}_row_id"]
    ).assign(label=0)

    # Sampling part
    if size is None or size > len(candidates):
        size = len(candidates)

    negatives = candidates.sample(size, random_state=random_state).reset_index(drop=True)

    ordered_cols = [
        f"{left_tag}_row_id",
        f"{right_tag}_row_id",
        f"title_{left_tag}",
        f"title_{right_tag}",
        f"platform_{left_tag}",
        f"platform_{right_tag}",
        f"release_year_{left_tag}",
        f"release_year_{right_tag}",
        "label"
    ]
    return negatives.assign(label=0)[ordered_cols]

pm_positive = build_positive_pairs(playtime, metacritic, "playtime", "metacritic")
ps_positive = build_positive_pairs(playtime, sales, "playtime", "sales")

pm_negative = build_negative_pairs_strict(playtime, metacritic, "playtime", "metacritic",size=1000)
ps_negative = build_negative_pairs_strict(playtime, sales, "playtime", "sales",size=1000)


In [24]:
from rapidfuzz import fuzz
import pandas as pd


def sim_title(a: str, b: str) -> float:
    return fuzz.token_set_ratio(a, b) / 100.0

def sim_publisher(a: str, b: str) -> float:
    return fuzz.token_set_ratio(a, b) / 100.0

def sim_developer(a: str, b: str) -> float:
    return fuzz.token_set_ratio(a, b) / 100.0
def sim_platform(a: str, b: str) -> float:
    return 1.0 if a == b else 0.0

def sim_year(y1: float, y2: float) -> float:
    diff = abs(y1 - y2)
    if diff == 0:
        return 1.0
    if diff == 1:
        return 0.8
    if diff == 2:
        return 0.6
    return 0.0

def overall_sim_ps(row):
    t   = sim_title(row["title_p"],      row["title_s"])
    pl  = sim_platform(row["platform_p"], row["platform_s"])
    y   = sim_year(row["release_year_p"], row["release_year_s"])
    pub = sim_publisher(row["publisher_p"], row["publisher_s"])
    return 0.45 * t + 0.25 * pl + 0.15 * y + 0.15 * pub

def overall_sim_pm(row):
    """
    Overall similarity for playtime–metacritic pair:
    uses title, platform, release_year, and developer.
    """
    t   = sim_title(row["title_p"],       row["title_m"])
    pl  = sim_platform(row["platform_p"], row["platform_m"])
    y   = sim_year(row["release_year_p"], row["release_year_m"])
    dev = sim_developer(row["developer_p"], row["developer_m"])

    return 0.45 * t + 0.25 * pl + 0.15 * y + 0.15 * dev

# ---- corner case builder for playtime–sales ----
def build_ps_corner_cases(playtime_df, sales_df, threshold=0.85):
    """
    1. Block on platform + release_year to create pair_candidates.
    2. Compute overall similarity for each candidate.
    3. Keep only rows with similarity >= threshold.
    4. Add empty 'label' column for later manual annotation.
    """

    # select + rename columns with p_ / s_ prefixes
    p = (
        playtime_df[
            ["playtime_row_id", "title", "platform", "release_year", "publisher"]
        ]
        .dropna(subset=["title", "platform", "release_year", "publisher"])
        .rename(columns={
            "playtime_row_id": "p_row_id",
            "title": "title_p",
            "platform": "platform_p",
            "release_year": "release_year_p",
            "publisher": "publisher_p",
        })
    )

    s = (
        sales_df[
            ["sales_row_id", "title", "platform", "release_year", "publisher"]
        ]
        .dropna(subset=["title", "platform", "release_year", "publisher"])
        .rename(columns={
            "sales_row_id": "s_row_id",
            "title": "title_s",
            "platform": "platform_s",
            "release_year": "release_year_s",
            "publisher": "publisher_s",
        })
    )

    # 1) BLOCKING: only pair rows with same platform + release_year
    pair_candidates = p.merge(
        s,
        left_on=["platform_p", "release_year_p"],
        right_on=["platform_s", "release_year_s"],
        how="inner",
    )

    # 2) SIMILARITY
    pair_candidates["similarity"] = pair_candidates.apply(overall_sim_ps, axis=1)

    # 3) FILTER by threshold
    corner_df = pair_candidates[
    (pair_candidates["similarity"] < 1.0) &
    (pair_candidates["similarity"] >= threshold)
].copy()

    # 4) LABEL column (empty / NaN)
    corner_df["label"] = pd.NA

    # optional: choose column order
    ordered_cols = [
        "p_row_id", "s_row_id",
        "title_p", "title_s",
        "platform_p", "platform_s",
        "release_year_p", "release_year_s",
        "publisher_p", "publisher_s",
        "similarity", "label",
    ]
    return corner_df[ordered_cols]


def build_pm_corner_cases(playtime_df, metacritic_df, threshold=0.85):
    """
    1. Block on platform + release_year to create pair_candidates.
    2. Compute overall similarity for each candidate.
    3. Keep only rows with threshold <= similarity < 1.0.
    4. Add empty 'label' column for later manual annotation.
    """

    # select + rename columns with p_ / m_ prefixes
    p = (
        playtime_df[
            ["playtime_row_id", "title", "platform", "release_year", "developer"]
        ]
        .dropna(subset=["title", "platform", "release_year", "developer"])
        .rename(columns={
            "playtime_row_id": "p_row_id",
            "title": "title_p",
            "platform": "platform_p",
            "release_year": "release_year_p",
            "developer": "developer_p",
        })
    )

    m = (
        metacritic_df[
            ["metacritic_row_id", "title", "platform", "release_year", "developer"]
        ]
        .dropna(subset=["title", "platform", "release_year", "developer"])
        .rename(columns={
            "metacritic_row_id": "m_row_id",
            "title": "title_m",
            "platform": "platform_m",
            "release_year": "release_year_m",
            "developer": "developer_m",
        })
    )

    # 1) BLOCKING: same platform + release_year
    pair_candidates = p.merge(
        m,
        left_on=["platform_p", "release_year_p"],
        right_on=["platform_m", "release_year_m"],
        how="inner",
    )

    # 2) SIMILARITY
    pair_candidates["similarity"] = pair_candidates.apply(overall_sim_pm, axis=1)

    # 3) FILTER: threshold <= similarity < 1.0
    corner_df = pair_candidates[
        (pair_candidates["similarity"] >= threshold) &
        (pair_candidates["similarity"] < 1.0)
    ].copy()

    # 4) LABEL column (empty / NaN)
    corner_df["label"] = pd.NA

    # optional: column order
    ordered_cols = [
        "p_row_id", "m_row_id",
        "title_p", "title_m",
        "platform_p", "platform_m",
        "release_year_p", "release_year_m",
        "developer_p", "developer_m",
        "similarity", "label",
    ]
    return corner_df[ordered_cols]

ps_corner = build_ps_corner_cases(playtime, sales, threshold=0.85)
pm_corner = build_pm_corner_cases(playtime,metacritic,threshold=0.85)

In [ ]:
pm_corner_a=pd.read_csv("labeled_corner_cases/pm_corner_labeled_a.csv")
pm_corner_b=pd.read_csv("labeled_corner_cases/pm_corner_labeled_b.csv")
pm_corner_b=pm_corner_b.iloc[:150,:12]
pm_corner = pd.concat(
    [pm_corner_a.reset_index(drop=True), pm_corner_b.reset_index(drop=True)],ignore_index=True,
)
ps_corner=pd.read_csv("labeled_corner_cases/ps_corner_labeled.csv")

In [39]:
print("Counts of labels in Playtime-Metacritic Pair Corner Cases:")
display(pm_corner.label.value_counts())

print("Counts of labels in Playtime-Sales Pair Corner Cases:")
display(ps_corner.label.value_counts())

Counts of labels in Playtime-Metacritic Pair Corner Cases:


label
1.0    275
0.0     25
Name: count, dtype: int64

Counts of labels in Playtime-Sales Pair Corner Cases:


label
1    278
0     22
Name: count, dtype: int64

In [ ]:
def compose_20_30_50(pos,neg,corners,
    total_n: int,
    *,
    seed: int = 42,
):
    """
    Build a labeled set with the classic 20/30/50 distribution:
      - 20% positives (exact ISBN matches)
      - 30% corners (text-similarity band; unlabeled -> to be labeled manually)
      - 50% negatives (random pairs avoiding equal ISBNs)
    Returns: (pos_df, corners_df, neg_df)
      pos_df: id_left, id_right, label=1
      corners_df: id_left, id_right, title_left, author_left, title_right, author_right, sim, why, label=NA
      neg_df: id_left, id_right, label=0
    """
    n_pos = int(round(total_n * 0.20))
    n_cor = int(round(total_n * 0.30))
    n_neg = total_n - n_pos - n_cor

    # 1) full pool of positives, then sample
    pos_sample = pos.sample(n=min(n_pos, len(pos)), random_state=seed) if len(pos) else pos

    # 2) full set of corners (banded), then sample
    corners_sample = corners.sample(n=min(n_cor, len(corners)), random_state=seed) if len(corners) else corners

    # 3) sample negatives directly
    neg_sample = neg.sample(n=min(n_neg, len(neg)), random_state=seed) if len(neg) else neg


    return pos_sample.reset_index(drop=True), corners_sample.reset_index(drop=True), neg_sample.reset_index(drop=True)

pos_PS,corners_PS,neg_PS=compose_20_30_50(ps_positive,ps_negative,ps_corner,total_n=1000)
pos_PM,corners_PM,neg_PM=compose_20_30_50(pm_positive,pm_negative,pm_corner,total_n=1000)

In [51]:
rename_ps = {"playtime_row_id": "id_left", "sales_row_id": "id_right"}
pos_PS, neg_PS = [df.rename(columns=rename_ps) for df in (pos_PS, neg_PS)]
rename_pm = {"playtime_row_id": "id_left", "metacritic_row_id": "id_right"}
pos_PM, neg_PM = [df.rename(columns=rename_pm) for df in (pos_PM, neg_PM)]

corners_PS.rename(columns={"p_row_id": "id_left", "s_row_id": "id_right"}, inplace=True)
corners_PM.rename(columns={"p_row_id": "id_left", "m_row_id": "id_right"}, inplace=True)


In [52]:
def finalize_and_split(pos_df, corners_labeled_df, neg_df, *, test_frac=0.2, seed=42):
    """
    Merge positives + labeled corners + negatives, then stratified split (train/test only).
    Returns:
        train_df, test_df  (each with id_left, id_right, label)
    """
    # Merge all labeled pairs
    all_labeled = pd.concat(
        [
            pos_df[["id_left", "id_right", "label"]],
            corners_labeled_df[["id_left", "id_right", "label"]],
            neg_df[["id_left", "id_right", "label"]],
        ],
        ignore_index=True
    ).dropna(subset=["label"]).drop_duplicates()

    # Ensure binary integer labels
    all_labeled["label"] = all_labeled["label"].astype(int)

    # Shuffle once for randomness
    L = all_labeled.sample(frac=1.0, random_state=seed).reset_index(drop=True)

    # Stratified split (preserve 0/1 balance)
    parts = {"train": [], "test": []}
    for y in (0, 1):
        grp = L[L["label"] == y]
        n = len(grp)
        n_test = int(round(n * test_frac))
        test_part = grp.iloc[:n_test]
        train_part = grp.iloc[n_test:]
        parts["train"].append(train_part)
        parts["test"].append(test_part)

    train = pd.concat(parts["train"], ignore_index=True)
    test  = pd.concat(parts["test"],  ignore_index=True)
    return train, test

In [54]:
train_PS, test_PS = finalize_and_split(pos_PS, corners_PS, neg_PS)
train_PM, test_PM = finalize_and_split(pos_PM, corners_PM, neg_PM)

In [ ]:
import pandas as pd

counts = pd.concat(
    {
        "train_PS": train_PS.label.value_counts(),
        "test_PS":  test_PS.label.value_counts(),
        "train_PM": train_PM.label.value_counts(),
        "test_PM":  test_PM.label.value_counts(),
    },
    axis=1,
).fillna(0).astype(int).T  # rows = splits, cols = labels

print(counts.to_markdown())   # or display(counts) in a notebook

|          |   0 |   1 |
|:---------|----:|----:|
| train_PS | 418 | 378 |
| test_PS  | 104 |  95 |
| train_PM | 420 | 375 |
| test_PM  | 105 |  94 |


In [66]:
from pathlib import Path

out_dir = Path(ROOT) / MLDS_DIR
try:
    out_dir.mkdir(parents=True, exist_ok=True)
except OSError as e:
    raise RuntimeError(f"Cannot create output dir {out_dir}") from e

for name, df in {
    "train_PS.parquet": train_PS,
    "train_PM.parquet": train_PM,
    "test_PS.parquet":  test_PS,
    "test_PM.parquet":  test_PM,
}.items():
    df.to_parquet(out_dir / name)